In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import re

# Decryption function
def decode_cf_email(encoded_email):
    decoded_email = ""
    k = int(encoded_email[:2], 16)

    for i in range(2, len(encoded_email), 2):
        decoded_email += chr(int(encoded_email[i:i+2], 16) ^ k)

    return decoded_email

# Function to fetch company emails
def fetch_company_emails():
    base_url = "https://www.teknoparkistanbul.com.tr"
    url = "https://www.teknoparkistanbul.com.tr/firmalar"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
                      '(KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    try:
        # Fetch HTML content of the main page
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise HTTPError for bad responses
    except requests.exceptions.RequestException as e:
        print(f"Failed to retrieve the webpage: {e}")
        return

    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all company links
    company_links = soup.find_all('a', href=re.compile(r'/firmalar/[\w-]+'))

    company_emails = []
    count = 0
    for link in company_links:
        company_relative_url = link['href']
        company_url = urljoin(base_url, company_relative_url)  # Correctly construct the company URL

        try:
            # Fetch HTML content of the company page
            company_response = requests.get(company_url, headers=headers)
            company_response.raise_for_status()
        except requests.exceptions.RequestException as e:
            print(f"Failed to retrieve company page {company_url}: {e}")
            continue

        company_soup = BeautifulSoup(company_response.content, 'html.parser')

        # Find all email elements containing encoded emails
        email_elements = company_soup.find_all('span', class_='__cf_email__')

        count = count + 1
        for email_element in email_elements:
            encoded_email = email_element['data-cfemail']  # Extract the encoded part
            decoded_email = decode_cf_email(encoded_email)
            company_emails.append(decoded_email)
            print(decoded_email)
    print(f"{count} tane email alindi")

    # Remove duplicates
    company_emails = list(set(company_emails))

    # Write emails to a text file
    with open('company_emails.txt', 'w') as file:
        for email in company_emails:
            file.write(email + '\n')

    print(f"Successfully extracted {len(company_emails)} email addresses.")

# Execute the function
fetch_company_emails()


In [3]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("gmail_address")
secret_value_1 = user_secrets.get_secret("gmail_password")

In [4]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email import encoders
import os

def send_email(sender_email, sender_password, receiver_email, subject, body, attachment_path):
    # Establishing connection with Gmail's SMTP server
    smtp_server = 'smtp.gmail.com'
    smtp_port = 587
    server = smtplib.SMTP(smtp_server, smtp_port)
    server.starttls()
    server.login(sender_email, sender_password)
    
    # Creating email message
    message = MIMEMultipart()
    message['From'] = "Mehmet Sait Cubukcu"
    message['To'] = receiver_email
    message['Subject'] = subject
    
    # Attaching body to the email
    message.attach(MIMEText(body, 'plain'))
    
    # Attaching the resume as an attachment
    filename = os.path.basename(attachment_path)
    attachment = open(attachment_path, "rb")
    
    part = MIMEBase('application', 'octet-stream')
    part.set_payload(attachment.read())
    encoders.encode_base64(part)
    part.add_header('Content-Disposition', f'attachment; filename= {filename}')
    
    message.attach(part)
    text = message.as_string()
    
    # Sending the email
    server.sendmail(sender_email, receiver_email, text)
    
    # Closing the connection
    server.quit()

# Example usage:
if __name__ == "__main__":
    sender_email = secret_value_0
    sender_password = secret_value_1
    receiver_email = "akademi282834@gmail.com"
    subject = "Job Application"
    body = """Dear HR Manager,

I am a recent graduate of Computer Engineering from Konya Technical University, having completed my studies in June 2024.

I have a keen interest in machine learning, natural language processing and computer vision fields. I develop projects and participate in competitions within these areas. I am seeking a position where I can further develop myself and gain valuable experiences.

You can find my resume attached. I hope to hear from you.

Thank you for your attention and time.

Best regards,
Mehmet Sait Cubukcu
+90 551 192 4193     """
    
attachment_path = "/kaggle/input/myresume/M Sait Cubukcu Resume.pdf"  # Replace with your resume file path
    
#send_email(sender_email, sender_password, receiver_email, subject, body, attachment_path)


In [1]:
company_emails_test = ["ysfkmlpinarci@gmail.com","akademi282834@gmail.com","m4257066@gmail.com"]
with open('company_emails_test.txt', 'w') as file:
    for email in company_emails_test:
        file.write(email + '\n')

In [25]:
sent_companies = []

In [31]:
with open('/kaggle/input/left-companies/company_emails.txt', 'r') as file:
    for company_email in file:
        if company_email in sent_companies:
            continue
            print(f"Zaten {company_email} sirketine mail gonderildi")
        else:
            send_email(sender_email, sender_password, company_email, subject, body, attachment_path)
            print(f"{company_email} sirketine email basariyla gonderildi")
            sent_companies.append(company_email)


info@techsign.com.tr
 sirketine email basariyla gonderildi
contact@palamartech.com
 sirketine email basariyla gonderildi
info@semper-tech.com
 sirketine email basariyla gonderildi
info@lostar.com
 sirketine email basariyla gonderildi
info@secromix.com
 sirketine email basariyla gonderildi
info@eclipse.com.tr
 sirketine email basariyla gonderildi
info@pangolinlab.com
 sirketine email basariyla gonderildi
nazbiyoteknoloji34@gmail.com
 sirketine email basariyla gonderildi
info@tuyar.com.tr
 sirketine email basariyla gonderildi
info@regiusbiotech.com
 sirketine email basariyla gonderildi
info@eczacibasi.com.tr
 sirketine email basariyla gonderildi
suimc-info@sabanciuniv.edu
 sirketine email basariyla gonderildi
info@blitzsystems.com.tr
 sirketine email basariyla gonderildi
info.tr@zennodefence.com
 sirketine email basariyla gonderildi
info@idealab.com.tr
 sirketine email basariyla gonderildi
thefuturenatural20@gmail.com
 sirketine email basariyla gonderildi
bilgi@gelisimpark.com.tr
 sirket